In [ ]:
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

data_train = pd.read_csv('train_2008.csv')
dft = pd.read_csv('test_2012.csv')
print(dft.shape)
#remove = ['id', 'HRMONTH', 'HRYEAR4']
#dft = dft.drop(remove, axis=1)
#data_train = data_train.drop(remove, axis=1)
y = data_train['target']
df = data_train.drop(['target'], axis = 1)

test = pd.concat([df['GESTFIPS'], y], axis=1)
test2 = test[test['target'] != 0]
test2 = test2.drop(columns=['target'])
test = test.drop(columns=['target'])
tes2 = test2['GESTFIPS'].value_counts()
tes1 = test['GESTFIPS'].value_counts()
result = (tes2 / tes1)
for i in range(60):
    if i in result.keys():
        df.loc[df['GESTFIPS'] == i, 'GESTFIPS'] = result[i] ** 2
        dft.loc[dft['GESTFIPS'] == i, 'GESTFIPS'] = result[i] ** 2
    else:
        df.loc[df['GESTFIPS'] == i, 'GESTFIPS'] = result.mean() ** 2
        dft.loc[dft['GESTFIPS'] == i, 'GESTFIPS'] = result.mean() ** 2
        

test = pd.concat([df['PEEDUCA'], y], axis=1)
test2 = test[test['target'] != 0]
test2 = test2.drop(columns=['target'])
test = test.drop(columns=['target'])
tes2 = test2['PEEDUCA'].value_counts()
tes1 = test['PEEDUCA'].value_counts()
result = (tes2 / tes1)

for i in range(60):
    if i in result.keys():
        df.loc[df['PEEDUCA'] == i, 'PEEDUCA'] = result[i] ** 2
        dft.loc[dft['PEEDUCA'] == i, 'PEEDUCA'] = result[i] ** 2
    else:
        df.loc[df['PEEDUCA'] == i, 'PEEDUCA'] = result.mean() ** 2
        dft.loc[dft['PEEDUCA'] == i, 'PEEDUCA'] = result.mean() ** 2 

#df = df.append(dft, ignore_index = True)
scaler = StandardScaler()
df[df.columns[:].tolist()] = scaler.fit_transform(df[df.columns[:].tolist()])

scaler = StandardScaler()
dft[dft.columns[:].tolist()] = scaler.fit_transform(dft[dft.columns[:].tolist()])

#X = df[df.columns[:].tolist()]
#y_train = y
#X_train = X.head(64667)
#X_test = X.tail(16000)

X_train = df[df.columns[:].tolist()]
y_train = y
X_test = dft[dft.columns[:].tolist()]
print(X_test.shape)

X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train.values, y_train.values, test_size=0.2, random_state=123)

# Create model here given constraints in the problem
model = Sequential()

model.add(Dense(1024))
model.add(Activation('relu'))
# model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.30))
model.add(Dense(600))
model.add(Activation('relu'))
# model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.20))
model.add(Dense(50))
model.add(Activation('relu'))
# model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.20))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

fit = model.fit(X_train1, y_train1, batch_size=170, nb_epoch=20)
model.summary()
pred = model.predict(X_test1)
pred_test = model.predict(X_test)
print(pred_test.shape)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score
import csv

n_estimators_optimal = -1

data_train = pd.read_csv('train_2008.csv')
dft = pd.read_csv('test_2012.csv')
#remove = ['id', 'HRMONTH', 'HRYEAR4']
#dft = dft.drop(remove, axis=1)
#data_train = data_train.drop(remove, axis=1)
y = data_train['target']
df = data_train.drop(['target'], axis = 1)

#for c in to_remove:
#    cols.remove(c)

cols = ["HUFINAL", "HETENURE", "HRHTYPE", "GEREG", "GESTCEN", "GTCBSAST", "GTCSA", "PERRP", "GESTFIPS", "PEEDUCA",
        'PEMARITL', 'PTDTRACE', 'PRDTHSP', 'PENATVTY', 'PEMNTVTY', 'PEFNTVTY', 'PUSLFPRX', 'PEHRRSN2', 'PRCHLD', 
        'PEIO1COW', 'PUIO1MFG', 'PRDTCOW1', 'PRDTIND1', 'PRDTOCC1', 'HEHOUSUT']

for col in cols:
    test = pd.concat([df[col], y], axis=1)
    test2 = test[test['target'] != 0]
    test2 = test2.drop(columns=['target'])
    test = test.drop(columns=['target'])
    tes2 = test2[col].value_counts()
    tes1 = test[col].value_counts()
    result = (tes2 / tes1)
    for i in range(60):
        if i in result.keys():
            df.loc[df[col] == i, col] = result[i] ** 2
            dft.loc[dft[col] == i, col] = result[i] ** 2
        else:
            df.loc[df[col] == i, col] = result.mean() ** 2
            dft.loc[dft[col] == i, col] = result.mean() ** 2
    
    
scaler = StandardScaler()
df[df.columns[:].tolist()] = scaler.fit_transform(df[df.columns[:].tolist()])

scaler = StandardScaler()
dft[dft.columns[:].tolist()] = scaler.fit_transform(dft[dft.columns[:].tolist()])


#X = df[df.columns[:].tolist()]
#y_train = y
#X_train = X.head(64667)
#X_test = X.tail(16000)

X_train = df[df.columns[:].tolist()]
y_train = y
X_test = dft[dft.columns[:].tolist()]
print(X_test.shape)

def modelfit(alg, X, y,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):

    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X.values, label=y.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, verbose_eval = 500)
        alg.set_params(n_estimators=cvresult.shape[0])
        print(cvresult.shape[0])
        n_estimators_optimal = cvresult.shape[0]

    alg.fit(X, y,eval_metric='auc')

    dtrain_predictions = alg.predict(X)
    dtrain_predprob = alg.predict_proba(X)[:,1]

    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(y.values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(y, dtrain_predprob))

    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

alg = XGBClassifier(learning_rate=0.1, n_estimators=205, max_depth=5,
                        min_child_weight=3, gamma=0.2, subsample=0.8, colsample_bytree=0.6,

                        objective='binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
alg.fit(X_train, y_train)
pred_prob = 0.65*alg.predict_proba(X_test)[: ,1]
pred_prob2 = 0.35*pred_test
total_pred = pred_prob + pred_prob2[0]
with open('output3_30.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id,target'])
    for i in range(len(total_pred)):
        writer.writerow([str(i) + ',' + str(total_pred[i])])